In [ ]:
import psycopg2, io, base64
from flask import Flask, render_template, request, url_for, redirect
import pandas as pd, matplotlib.pyplot as plt, plotly, json
import plotly.express as px

app = Flask(__name__)

def bd(sql):
    conn = psycopg2.connect(host='localhost', database='IPA1', user='postgres', password='19122022')
    cur = conn.cursor()
    cur.execute(sql)
    dados = cur.fetchall()
    cur.close()
    conn.close()
    return dados

@app.route('/', methods=('GET', 'POST'))
def autentic():
    if request.method == 'POST':
        conta = request.form['login']
        senha = request.form['senha']
        sql="select * from administrador.usuario where (login='"+conta+"' and senha='"+senha+"')"
        #sql="select * from administrador.usuario where (conta='"+conta+"')"
        record=bd(sql)
        nr=len(record)
        if nr>=1:
            return redirect(url_for('home'))
        else:
            return redirect(url_for('indexnulo'))
    return render_template('autentic.html')

@app.route('/nulo')
def indexnulo():
    return render_template('indexnulo.html')


@app.route('/home')
def home():
    sql = '''
    SELECT m.id_menu, m.name_menu, s.id_submenu, s.name_submenu
    FROM menu m
    LEFT JOIN submenu s ON s.id_menu = m.id_menu
    ORDER BY m.ordem, s.id_submenu;
    '''
    dados = bd(sql)

    menu_dict = {}

    for id_menu, name_menu, id_submenu, name_submenu in dados:
        if name_menu not in menu_dict:
            menu_dict[name_menu] = []
        
        if name_submenu:
            menu_dict[name_menu].append(name_submenu)
    
    return render_template('homepage.html', menus=menu_dict)





@app.route('/t1')
def index():
    sql='select * from menu'
    menus= bd(sql)
    return render_template('index.html', menus=menus)

@app.route('/t2')
def index1():
    sql='select * from submenu'
    menus= bd(sql)
    return render_template('index1.html', menus=menus)

@app.route('/bi')
def index2():
    return render_template('index2.html')

@app.route('/mapa')
def index3():
    sql = '''
        SELECT c.nome_crime, COUNT(co.id_ocorrencia) AS total_ocorrencias
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        GROUP BY c.nome_crime
        ORDER BY total_ocorrencias DESC;
    '''
    dados = bd(sql)

    nomes_crimes = [linha[0] for linha in dados]
    totais = [linha[1] for linha in dados]

    df = pd.DataFrame({
        'Crime': nomes_crimes,
        'Total': totais
    })

    # Criar o gráfico
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Crime'], df['Total'], color='skyblue')
    ax.set_title('Quantidade de Ocorrências por Crime')
    ax.set_xlabel('Tipo de Crime')
    ax.set_ylabel('Total de Ocorrências')
    plt.xticks(rotation=45)

    # Salvar imagem em memória
    img = io.BytesIO()
    plt.tight_layout()
    plt.savefig(img, format='png')
    img.seek(0)
    plt.close(fig)

    # Codificar imagem para base64
    plot_url = base64.b64encode(img.getvalue()).decode()

    return render_template('index3.html', plot_url=plot_url)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Jul/2025 09:42:19] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2025 09:42:29] "GET /bi HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2025 09:42:29] "GET /static/images/plot1.png HTTP/1.1" 304 -
127.0.0.1 - - [08/Jul/2025 09:42:33] "GET /mapa HTTP/1.1" 200 -
